# Final project guidelines

**Note:** Use these guidelines if and only if you are pursuing a **final project of your own design**. For those taking the final exam instead of the project, see the (separate) final exam notebook.

### The task

Your task is to: identify an interesting problem connected to the humanities or humanistic social sciences that's addressable with the help of computational methods, formulate a hypothesis about it, devise an experiment or experiments to test your hypothesis, present the results of your investigations, and discuss your findings.

These tasks essentially replicate the process of writing an academic paper. You can think of your project as a paper in miniature.

You are free to present each of these tasks as you see fit. You should use narrative text (that is, your own writing in a markdown cell), citations of others' work, numerical results, tables of data, and static and/or interactive visualizations as appropriate. Total length is flexible and depends on the number of people involved in the work, as well as the specific balance you strike between the ambition of your question and the sophistication of your methods. But be aware that numbers never, ever speak for themselves. Quantitative results presented without substantial discussion will not earn high marks. 

Your project should reflect, at minimum, ten **or more** hours of work by each participant, though you will be graded on the quality of your work, not the amount of time it took you to produce it. Most high-quality projects represent twenty or more hours of work by each member.

#### Pick an important and interesting problem!

No amount of technical sophistication will overcome a fundamentally uninteresting problem at the core of your work. You have seen many pieces of successful computational humanities research over the course of the semester. You might use these as a guide to the kinds of problems that interest scholars in a range of humanities disciplines. You may also want to spend some time in the library, reading recent books and articles in the professional literature. **Problem selection and motivation are integral parts of the project.** Do not neglect them.

### Format

You should submit your project as a PDF document created using the included $\LaTeX{}$ template. Consult the template for information on formatting and what is expected in each section. You can use your favorite text editor or something like [Overleaf](https://www.overleaf.com/) to edit this document. You will also submit this Jupyter notebook, along with all data necessary to reproduce your analysis. If your dataset is too large to share easily, let us know in advance so that we can find a workaround. 

All code used in the project should be present in the notebook (except for widely-available libraries that you import), but **be sure that we can read and understand your report in full without rerunning the code**. 

Because you are submitting essentially a mini-paper in the PDF writeup, I don't have any particular formatting expections for written material in this notebook. However, you should include **all code used when completing the final project, with comments added for clarity**. It should be straightforward to map code from the notebook to sections/figures/results in your paper, and vice versa.

### Grading

This project takes the place of the take-home final exam for the course. It is worth 35% of your overall grade. You will be graded on the quality and ambition of each aspect of the project. No single component is more important than the others.

### Practical details

* The project is due at **4:30 PM EST on Wednesday, December 17** via upload to CMS of a single zip file containing your fully executed Jupyter notebook report, a PDF copy of the notebook, and all associated data. **You may not use slip days for the final project or exam**. 
* You may work alone or in a group of up to three total members.
    * If you work in a group, be sure to list the names of the group members.
    * For groups, create your group on CMS and submit one notebook for the entire group. **Each group should also submit a statement of responsibility** that describes in general terms who performed which parts of the project.
* You may post questions on Ed, but should do so privately (visible to course staff only).

---

## Your info
* NetID(s): rr772, aa2289, kx63
* Name(s): Razika Rahman, Afran Ahmed, Kyleena Xin
---

In [58]:
# imports go here
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split

In [53]:
data = pd.read_csv("data/tedtalk_sample.csv")

data.head()

data_y = data['views']
data_for_vec = data.drop(columns=['views'])

In [54]:
#Vectorizer

vectorizer = TfidfVectorizer(
    input = "content",
    encoding = "utf-8",
    stop_words = "english",
    strip_accents= "unicode",
    min_df= 2,
    max_df = 0.5,
    norm = 'l2',
    use_idf=True,
    ngram_range= (1,2),
)

#fit vectorizer onto corpus
data_vectorized = vectorizer.fit_transform(data_for_vec['transcript'])
print(f"Shape of Feature Matrix: {data_vectorized.shape}")

#store feature names + values
feature_name = vectorizer.get_feature_names_out()
val = data_vectorized.mean(axis = 0).A1

top_features = pd.DataFrame({
    "feature": feature_name,
    "value": val
}).sort_values("value", ascending = False)

top_features.head(30)

Shape of Feature Matrix: (1000, 88944)


,feature,value
86118,women,0.015956
56620,percent,0.012327
34786,human,0.011975
16977,data,0.011578
75727,technology,0.011132
73113,story,0.010398
46537,love,0.009998
67213,school,0.009985
51125,music,0.009900
71274,space,0.009865


In [60]:
#Regression
glr = LinearRegression(n_jobs = -1)

X_train, X_test, y_train, y_test = train_test_split(data_vectorized, data_y,
    test_size = 0.2, random_state = 42
)

glr.fit(X_train, y_train)
y_pred = glr.predict(X_test)
print(f"R2 Score: {r2_score(y_test, y_pred)}")
print(f"MSE: {mean_squared_error(y_test, y_pred)}")

R2 Score: 0.017258789010596076
MSE: 3035115924112.2812
